# Data Extraction

In [5]:
import os
import requests
import time
from pymongo import MongoClient
from tqdm import tqdm
from dotenv import load_dotenv

# Load environment variables
load_dotenv(dotenv_path=r"C:\Users\madhu\OneDrive\Desktop\Project - Git Hub Recommendation\Git-Hub-Recommendation\password.env")  # Use your absolute path
GITHUB_TOKEN = os.getenv("GITHUB_API_KEY")
MONGO_URI = os.getenv("MONGO_URI")

if not GITHUB_TOKEN:
    raise Exception("❌ GitHub API key is missing!")
else:
    print("✅ GitHub API key loaded successfully. Ready to access GitHub API.")

if not MONGO_URI:
    raise Exception("❌ MongoDB URI is missing!")
else:
    print("✅ MongoDB URI loaded successfully. Ready to access MongoDB.")

# Headers for GitHub API
HEADERS = {
    "Authorization": f"token {GITHUB_TOKEN}",
    "Accept": "application/vnd.github.v3+json"
}

# MongoDB setup
client = MongoClient(MONGO_URI)
db = client["github_recommendation_db"]
collection = db["Users__data"]

# Helper functions
def get_json(url):
    response = requests.get(url, headers=HEADERS)
    if response.status_code == 403:  # Rate limit exceeded
        print("⚠️ Rate limit hit. Waiting for 60 seconds...")
        time.sleep(60)
        response = requests.get(url, headers=HEADERS)
    response.raise_for_status()
    return response.json()

def get_users_batch(since=0, per_page=100):
    url = f"https://api.github.com/users?since={since}&per_page={per_page}"
    return get_json(url)

def get_user_details(username):
    url = f"https://api.github.com/users/{username}"
    return get_json(url)

def get_list(url, key='login'):
    return [item[key] for item in get_json(url)]

def get_starred_or_subs(url):
    return [item['html_url'] for item in get_json(url)]

def get_languages(username):
    repos = get_json(f"https://api.github.com/users/{username}/repos")
    language_data = {}
    for repo in repos:
        langs = get_json(repo['languages_url'])
        for lang, count in langs.items():
            language_data[lang] = language_data.get(lang, 0) + count
    return language_data

def get_total_commits(username):
    repos = get_json(f"https://api.github.com/users/{username}/repos")
    total = 0
    for repo in repos:
        if repo.get('fork'):
            continue
        url = repo['commits_url'].replace("{/sha}", "")
        commits = requests.get(url, headers=HEADERS, params={'per_page': 1}).json()
        if isinstance(commits, list):
            total += len(commits)
    return total

def collect_and_store_user(username):
    try:
        print(f"🔍 Collecting data for user: {username}")
        user = get_user_details(username)
        data = {
            "Login": username,
            "Name": user.get("name"),
            "Bio": user.get("bio"),
            "Public Repositories": user.get("public_repos"),
            "Followers Count": user.get("followers"),
            "Following Count": user.get("following"),
            "Created At": user.get("created_at"),
            "Updated At": user.get("updated_at"),
            "Avatar URL": user.get("avatar_url"),
            "Profile URL": user.get("html_url"),
            "Followers List": get_list(f"https://api.github.com/users/{username}/followers"),
            "Following List": get_list(f"https://api.github.com/users/{username}/following"),
            "Starred Repositories": get_starred_or_subs(f"https://api.github.com/users/{username}/starred"),
            "Subscriptions": get_starred_or_subs(f"https://api.github.com/users/{username}/subscriptions"),
            "Organizations": get_list(f"https://api.github.com/users/{username}/orgs"),
            "Languages": get_languages(username),
            "Total Commits": get_total_commits(username)
        }
        collection.update_one({"Login": username}, {"$set": data}, upsert=True)
        print(f"✅ Data stored/updated for user: {username}")
    except Exception as e:
        print(f"❌ Error collecting data for {username}: {e}")

def batch_process_users(start_since=0, batches=1, per_page=10):
    since = start_since
    for _ in range(batches):
        users = get_users_batch(since, per_page)
        if not users:
            print("⚠️ No more users found. Ending batch processing.")
            break
        for user in tqdm(users, desc="Processing batch"):
            collect_and_store_user(user['login'])
        since = users[-1]['id']

def fetch_user_if_not_exists(username):
    if collection.find_one({"Login": username}):
        print(f"✅ User {username} already exists in DB.")
    else:
        print(f"🔍 Fetching data for missing user: {username}")
        collect_and_store_user(username)

# === Entry Point ===
if __name__ == "__main__":
    print("🚀 Starting GitHub user data collection...")
    batch_process_users(start_since=0, batches=2, per_page=50)
    print("🎉 Data collection complete!")



✅ GitHub API key loaded successfully. Ready to access GitHub API.
✅ MongoDB URI loaded successfully. Ready to access MongoDB.
🚀 Starting GitHub user data collection...


Processing batch:   0%|          | 0/50 [00:00<?, ?it/s]

🔍 Collecting data for user: mojombo


Processing batch:   2%|▏         | 1/50 [00:32<26:24, 32.34s/it]

✅ Data stored/updated for user: mojombo
🔍 Collecting data for user: defunkt


...

✅ Data stored/updated for user: dkubb
🔍 Collecting data for user: jnicklas


Processing batch:  98%|█████████▊| 49/50 [23:20<00:30, 30.05s/it]

✅ Data stored/updated for user: jnicklas
🔍 Collecting data for user: richcollins


Processing batch: 100%|██████████| 50/50 [23:47<00:00, 28.54s/it]

✅ Data stored/updated for user: richcollins
🎉 Data collection complete!
